In [30]:
#!/disk/scratch/mlp/miniconda2/bin/python
import os
import tensorflow as tf
import numpy as np

# Utilities
import datetime
import time
import pickle
import matplotlib.pyplot as plt

In [31]:
#check necessary environment variables are defined
assert 'MLP_DATA_DIR' in os.environ, 'An environment variable MLP_DATA_DIR must be set to the path containing MLP data before running script.'
# assert 'OUTPUT_DIR' in os.environ, 'An environment variable OUTPUT_DIR must be set to the path to write output to before running script.'
save_point = '/home/s1687487/0307/3/exp_2/' #'/home/s1687487/0228/0/exp/'

In [32]:
# Import data
# batch_size = 500
# from tensorflow.examples.tutorials.mnist import input_data
# mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

batch_size = 128
from mlp.data_providers import MSD10GenreDataProvider, MSD25GenreDataProvider
train_data = MSD10GenreDataProvider('train', batch_size = batch_size, max_num_batches=2)
valid_data = MSD10GenreDataProvider('valid', batch_size = batch_size, max_num_batches=1)

In [33]:
# Parameters
learning_rate = 0.001
# training_iters = 100000
training_epochs = 2
display_step = 5

# Network Parameters
n_input = 25 # MNIST data input (img shape: 28*28)
n_steps = 120 # timesteps
n_classes = 10 # MNIST total classes (0-9 digits)

In [34]:
def experiment():    
    # tf Graph input
    x = tf.placeholder("float", [None, n_steps, n_input])
    y = tf.placeholder("float", [None, n_classes])

    # Define weights
    weights = {
        'out': tf.Variable(tf.random_normal([n_hidden, n_classes]))
    }
    biases = {
        'out': tf.Variable(tf.random_normal([n_classes]))
    }

    tf.add_to_collection('vars', weights['out'])
    tf.add_to_collection('vars', biases['out'])
    saver = tf.train.Saver()

    def ConvNetwork(x, weights, biases):
        # Convolutional Layer #1
#         print 'x shape =', x.get_shape()
        input_layer = tf.reshape(x, [-1, n_steps, n_input, 1])
#         print 'input_layer shape =', input_layer.get_shape()
        conv1 = tf.contrib.layers.conv2d(input_layer,filters1_num,kernel1_size,padding='SAME')#,activation=tf.nn.elu)        
#         print 'conv1 shape =', conv1.get_shape()
        # Pooling Layer #1
        pool1 = tf.contrib.layers.max_pool2d(conv1, pool1_size, [2,1])
#         print 'pool1 shape =', pool1.get_shape()
        
        
        # Conv #2
#x shape = (?, 120, 25)
# input_layer shape = (?, 120, 25, 1)
# conv1 shape = (?, 120, 25, 32)
# pool1 shape = (?, 60, 25, 32)  

        conv2 = tf.contrib.layers.conv2d(pool1,filters2_num,kernel2_size,padding='VALID')#,activation=tf.nn.elu)        
#         print 'conv2 shape =', conv2.get_shape()
        pool2 = tf.contrib.layers.max_pool2d(conv2, pool2_size, [3,1])
#         print 'pool2 shape =', pool2.get_shape()
                        
        # Dense Layer
        reduc_t, reduc_inp, n_ch = pool2.get_shape()[1:4].as_list()
        multiple = reduc_t * reduc_inp * n_ch
        pool_flat = tf.reshape(pool2, [-1, multiple])
#         print 'pool1_flat shape =', pool1_flat.get_shape()
        
        w_dense = tf.Variable(
            tf.truncated_normal([multiple, n_units1], stddev=2. / (multiple + n_units1)**0.5), 
            'w_dense')
        b_dense = tf.Variable(tf.zeros([n_units1]), 'b_dense')
    
        dense = tf.nn.relu(tf.matmul(pool_flat, w_dense) + b_dense)
#         print 'dense shape =', dense.get_shape()
        dropout = tf.contrib.layers.dropout(dense, keep_prob=1.- do1_p)
#         print 'dropout shape = ', dropout.get_shape()
        
        final1 = tf.matmul(dropout, weights)
#         print 'final1 shape = ', final1.get_shape()
        final = final1 + biases
#         print 'final shape = ', final.get_shape()
        
        return final

    pred = ConvNetwork(x, weights['out'], biases['out'])

    # Define loss and optimizer
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # Evaluate model
    correct_pred = tf.equal(tf.argmax(pred,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

    # Initializing the variables
    init = tf.global_variables_initializer()

    # Performance pickles
    err_tr = np.zeros(training_epochs)
    acc_tr = np.zeros(training_epochs)
    err_val = np.zeros(training_epochs)
    acc_val = np.zeros(training_epochs)

    with tf.Session() as sess:
        print("Optimization Starts! CNN_(depth,n_filter1,n_filter2,p_DO,n_hidden)=(2,{0},{1},{2},{3})".format(filters1_num,filters2_num,do1_p,n_hidden))
        sess.run(init) #initial run

        for e in range(training_epochs):                              
            if e == 0:
                start = time.time()
                num_batches_tr = 0
                num_batches_val = 0

            running_error = 0.
            running_accuracy = 0.
            for input_batch, target_batch in train_data:
                if e == 0:
                    num_batches_tr += 1
                input_batch = input_batch.reshape((batch_size,n_steps,n_input))

                _, batch_error, batch_acc = sess.run(
                    [optimizer, cost, accuracy], 
                    feed_dict={x: input_batch, y: target_batch})
                running_error += batch_error
                running_accuracy += batch_acc
  
            running_error /= num_batches_tr
            err_tr[e] = running_error
            running_accuracy /= num_batches_tr
            acc_tr[e] = running_accuracy

            if ((e+1) % display_step == 0) or (e == 0):
                end = time.time()
                print('End of epoch {0:02d}: err(train)={1:.2f} acc(train)={2:.2f} ,,, taking {3:.1f}(sec)'
                      .format(e + 1, running_error, running_accuracy, end-start))

                start = time.time()            

                valid_error = 0.
                valid_accuracy = 0.
                for input_batch, target_batch in valid_data:
                    if e == 0:
                        num_batches_val += 1
                    input_batch = input_batch.reshape((batch_size,n_steps,n_input))

                    batch_error, batch_acc = sess.run(
                        [cost, accuracy], 
                        feed_dict={x: input_batch, y: target_batch})
                    valid_error += batch_error
                    valid_accuracy += batch_acc
                valid_error /= num_batches_val
                err_val[e] = valid_error
                valid_accuracy /= num_batches_val
                acc_val[e] = valid_accuracy

                end = time.time()
                print('                 err(valid)={0:.2f} acc(valid)={1:.2f} ,,, taking {2:.1f}(sec) '
                       .format(valid_error, valid_accuracy, end-start))

                start = time.time()

        #Save model - save session        
        sname = "params_cnnDeep2_n_filter12,p_DO,n_hidden,,,"+ str([filters1_num,filters2_num,do1_p,n_hidden])
        saver.save(sess, sname)
#             saver.save(sess, save_point+sname)
        #...performance as well - pickle
        prf_dict = {"err_tr": err_tr, "err_val": err_val, "acc_tr": acc_tr, "acc_val": acc_val}
        fname = "cnnDeep2_performances_n_filter12,p_DO,n_hidden,,,"+ str([filters1_num,filters2_num,do1_p,n_hidden])
        pickle.dump(prf_dict, open(fname+".p","wb"))
#             pickle.dump(prf_dict, open(save_point+fname,"wb"))

        #Reset the data
        train_data.reset()
        valid_data.reset()
        print ''

    #Reset the graph per each iteration
    tf.reset_default_graph()

In [35]:
filters1_num_list = [32]#[6,12,24,36]
filters2_num_list = [128,32,16]

dim_t_list = [3]#[3,5,7,13]
dim_t2_list = [4]

do1_p_list = [0.,0.5]

pool1_size = [2,1]
pool2_size = [2,1]

n_hidden_list = [30,64,128,256]

for n_hidden in n_hidden_list:
    n_units1 = n_hidden
    for dim_t in dim_t_list:
        for dim_t2 in dim_t2_list:
            for filters1_num in filters1_num_list:
                for filters2_num in filters2_num_list:
                    for do1_p in do1_p_list:
                        kernel1_size = [dim_t,n_input]
                        kernel2_size = [dim_t2,n_input]
                        experiment()
print("Optimization Finished!")

Optimization Starts! CNN_(depth,n_filter1,n_filter2,p_DO,n_hidden)=(2,32,128,0.0,30)
End of epoch 01: err(train)=4.88 acc(train)=0.09 ,,, taking 1.3(sec)
                 err(valid)=3.79 acc(valid)=0.13 ,,, taking 0.2(sec) 


TypeError: cannot concatenate 'str' and 'list' objects